In [1]:
# 모듈 import
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/cascade_rcnn/cascade_rcnn_x101_64x4d_fpn_1x_coco.py')

root='../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2021
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/cascade_rcnn_x101_64x4d_fpn_1x_coco_trash'

# for faster r-cnn
#cfg.model.roi_head.bbox_head.num_classes = 10

# for cascade r-cnn
for bbox_head in cfg.model.roi_head.bbox_head:
    bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)

In [3]:
log_config = dict(
    interval=10,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(
            type='WandbLoggerHook',
            init_kwargs=dict(
                project='p-stage_object_detection',
                name='moon'))])
# for wandb
cfg.log_config = log_config

In [6]:
cfg.model.test_cfg.rpn.nms.type = 'soft_nms'

cfg.evaluation.metric = 'mAP'

In [9]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

# dataset 확인
datasets[0]

loading annotations into memory...
Done (t=0.25s)
creating index...
index created!


In [11]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

/opt/ml/detection/mmdetection/mmdet/core/anchor/builder.py:16: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  '``build_anchor_generator`` would be deprecated soon, please use '
2021-09-28 09:14:12,704 - mmcv - INFO - initialize ResNeXt with init_cfg {'type': 'Pretrained', 'checkpoint': 'open-mmlab://resnext101_64x4d'}
2021-09-28 09:14:12,706 - mmcv - INFO - load model from: open-mmlab://resnext101_64x4d
2021-09-28 09:14:12,707 - mmcv - INFO - Use load_from_openmmlab loader
2021-09-28 09:14:13,098 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2021-09-28 09:14:13,141 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2021-09-28 09:14:13,155 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'f

2021-09-28 09:14:14,618 - mmcv - INFO - 
backbone.layer2.0.downsample.0.weight - torch.Size([512, 256, 1, 1]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,618 - mmcv - INFO - 
backbone.layer2.0.downsample.1.weight - torch.Size([512]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,619 - mmcv - INFO - 
backbone.layer2.0.downsample.1.bias - torch.Size([512]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,620 - mmcv - INFO - 
backbone.layer2.1.conv1.weight - torch.Size([512, 512, 1, 1]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,620 - mmcv - INFO - 
backbone.layer2.1.bn1.weight - torch.Size([512]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,621 - mmcv - INFO - 
backbone.layer2.1.bn1.bias - torch.Size([512]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,621 - mmcv - INFO - 
backbo

2021-09-28 09:14:14,649 - mmcv - INFO - 
backbone.layer3.2.bn1.bias - torch.Size([1024]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,650 - mmcv - INFO - 
backbone.layer3.2.conv2.weight - torch.Size([1024, 16, 3, 3]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,650 - mmcv - INFO - 
backbone.layer3.2.bn2.weight - torch.Size([1024]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,651 - mmcv - INFO - 
backbone.layer3.2.bn2.bias - torch.Size([1024]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,652 - mmcv - INFO - 
backbone.layer3.2.conv3.weight - torch.Size([1024, 1024, 1, 1]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,652 - mmcv - INFO - 
backbone.layer3.2.bn3.weight - torch.Size([1024]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,653 - mmcv - INFO - 
backbone.layer3.2.bn3.bia

2021-09-28 09:14:14,680 - mmcv - INFO - 
backbone.layer3.8.bn1.weight - torch.Size([1024]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,680 - mmcv - INFO - 
backbone.layer3.8.bn1.bias - torch.Size([1024]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,681 - mmcv - INFO - 
backbone.layer3.8.conv2.weight - torch.Size([1024, 16, 3, 3]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,682 - mmcv - INFO - 
backbone.layer3.8.bn2.weight - torch.Size([1024]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,682 - mmcv - INFO - 
backbone.layer3.8.bn2.bias - torch.Size([1024]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,683 - mmcv - INFO - 
backbone.layer3.8.conv3.weight - torch.Size([1024, 1024, 1, 1]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,683 - mmcv - INFO - 
backbone.layer3.8.bn3.wei

2021-09-28 09:14:14,714 - mmcv - INFO - 
backbone.layer3.14.conv1.weight - torch.Size([1024, 1024, 1, 1]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,715 - mmcv - INFO - 
backbone.layer3.14.bn1.weight - torch.Size([1024]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,715 - mmcv - INFO - 
backbone.layer3.14.bn1.bias - torch.Size([1024]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,716 - mmcv - INFO - 
backbone.layer3.14.conv2.weight - torch.Size([1024, 16, 3, 3]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,717 - mmcv - INFO - 
backbone.layer3.14.bn2.weight - torch.Size([1024]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,718 - mmcv - INFO - 
backbone.layer3.14.bn2.bias - torch.Size([1024]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,719 - mmcv - INFO - 
backbone.layer3.14.

2021-09-28 09:14:14,755 - mmcv - INFO - 
backbone.layer3.19.bn3.bias - torch.Size([1024]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,756 - mmcv - INFO - 
backbone.layer3.20.conv1.weight - torch.Size([1024, 1024, 1, 1]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,757 - mmcv - INFO - 
backbone.layer3.20.bn1.weight - torch.Size([1024]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,758 - mmcv - INFO - 
backbone.layer3.20.bn1.bias - torch.Size([1024]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,758 - mmcv - INFO - 
backbone.layer3.20.conv2.weight - torch.Size([1024, 16, 3, 3]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,759 - mmcv - INFO - 
backbone.layer3.20.bn2.weight - torch.Size([1024]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,760 - mmcv - INFO - 
backbone.layer3.20.

2021-09-28 09:14:14,797 - mmcv - INFO - 
backbone.layer4.2.bn2.weight - torch.Size([2048]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,797 - mmcv - INFO - 
backbone.layer4.2.bn2.bias - torch.Size([2048]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,798 - mmcv - INFO - 
backbone.layer4.2.conv3.weight - torch.Size([2048, 2048, 1, 1]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,798 - mmcv - INFO - 
backbone.layer4.2.bn3.weight - torch.Size([2048]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,799 - mmcv - INFO - 
backbone.layer4.2.bn3.bias - torch.Size([2048]): 
PretrainedInit: load from open-mmlab://resnext101_64x4d 
 
2021-09-28 09:14:14,799 - mmcv - INFO - 
neck.lateral_convs.0.conv.weight - torch.Size([256, 256, 1, 1]): 
XavierInit: gain=1, distribution=uniform, bias=0 
 
2021-09-28 09:14:14,800 - mmcv - INFO - 
neck.lateral_convs.0.conv.bias

In [ ]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2021-09-28 09:14:18,935 - mmdet - INFO - Start running, host: root@2f4df6ced18c, work_dir: /opt/ml/detection/mmdetection/work_dirs/cascade_rcnn_x101_64x4d_fpn_1x_coco_trash
2021-09-28 09:14:18,937 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) OptimizerHook            

/opt/ml/detection/mmdetection/mmdet/core/anchor/anchor_generator.py:323: UserWarning: ``grid_anchors`` would be deprecated soon. Please use ``grid_priors`` 
  warnings.warn('``grid_anchors`` would be deprecated soon. '
/opt/ml/detection/mmdetection/mmdet/core/anchor/anchor_generator.py:360: UserWarning: ``single_level_grid_anchors`` would be deprecated soon. Please use ``single_level_grid_priors`` 
  '``single_level_grid_anchors`` would be deprecated soon. '
2021-09-28 09:14:36,450 - mmdet - INFO - Epoch [1][10/1221]	lr: 3.796e-04, eta: 3:49:34, time: 0.941, data_time: 0.237, memory: 5807, loss_rpn_cls: 0.6929, loss_rpn_bbox: 0.0547, s0.loss_cls: 2.5125, s0.acc: 36.9141, s0.loss_bbox: 0.0939, s1.loss_cls: 0.7847, s1.acc: 52.9150, s1.loss_bbox: 0.0195, s2.loss_cls: 1.0965, s2.acc: 0.6836, s2.loss_bbox: 0.0055, loss: 5.2601, grad_norm: 54.9518
2021-09-28 09:14:42,861 - mmdet - INFO - Epoch [1][20/1221]	lr: 7.792e-04, eta: 3:12:52, time: 0.641, data_time: 0.008, memory: 5807, loss_rpn_cls

2021-09-28 09:16:47,526 - mmdet - INFO - Epoch [1][210/1221]	lr: 8.372e-03, eta: 2:41:01, time: 0.647, data_time: 0.007, memory: 5807, loss_rpn_cls: 0.1094, loss_rpn_bbox: 0.0271, s0.loss_cls: 0.3565, s0.acc: 91.4355, s0.loss_bbox: 0.1582, s1.loss_cls: 0.1616, s1.acc: 92.6318, s1.loss_bbox: 0.1167, s2.loss_cls: 0.0570, s2.acc: 95.7812, s2.loss_bbox: 0.0370, loss: 1.0236, grad_norm: 3.8565
2021-09-28 09:16:54,075 - mmdet - INFO - Epoch [1][220/1221]	lr: 8.771e-03, eta: 2:40:45, time: 0.655, data_time: 0.008, memory: 5807, loss_rpn_cls: 0.2062, loss_rpn_bbox: 0.0839, s0.loss_cls: 0.4749, s0.acc: 90.6934, s0.loss_bbox: 0.2071, s1.loss_cls: 0.1652, s1.acc: 93.9209, s1.loss_bbox: 0.0933, s2.loss_cls: 0.0593, s2.acc: 96.2061, s2.loss_bbox: 0.0265, loss: 1.3164, grad_norm: 4.8878
2021-09-28 09:17:00,599 - mmdet - INFO - Epoch [1][230/1221]	lr: 9.171e-03, eta: 2:40:28, time: 0.652, data_time: 0.007, memory: 5807, loss_rpn_cls: 0.1997, loss_rpn_bbox: 0.0543, s0.loss_cls: 0.4666, s0.acc: 89.7021

2021-09-28 09:19:04,491 - mmdet - INFO - Epoch [1][420/1221]	lr: 1.676e-02, eta: 2:36:41, time: 0.667, data_time: 0.008, memory: 5807, loss_rpn_cls: 0.1022, loss_rpn_bbox: 0.0376, s0.loss_cls: 0.2835, s0.acc: 93.3105, s0.loss_bbox: 0.1008, s1.loss_cls: 0.1387, s1.acc: 93.2252, s1.loss_bbox: 0.0899, s2.loss_cls: 0.0611, s2.acc: 94.0080, s2.loss_bbox: 0.0454, loss: 0.8592, grad_norm: 2.6565
2021-09-28 09:19:10,794 - mmdet - INFO - Epoch [1][430/1221]	lr: 1.716e-02, eta: 2:36:24, time: 0.630, data_time: 0.007, memory: 5807, loss_rpn_cls: 0.0703, loss_rpn_bbox: 0.0288, s0.loss_cls: 0.2174, s0.acc: 94.8389, s0.loss_bbox: 0.0825, s1.loss_cls: 0.1069, s1.acc: 94.2578, s1.loss_bbox: 0.0760, s2.loss_cls: 0.0488, s2.acc: 94.6973, s2.loss_bbox: 0.0423, loss: 0.6730, grad_norm: 2.2480
2021-09-28 09:19:17,168 - mmdet - INFO - Epoch [1][440/1221]	lr: 1.756e-02, eta: 2:36:10, time: 0.637, data_time: 0.007, memory: 5807, loss_rpn_cls: 0.0785, loss_rpn_bbox: 0.0312, s0.loss_cls: 0.3314, s0.acc: 91.9873

2021-09-28 09:21:22,009 - mmdet - INFO - Epoch [1][630/1221]	lr: 2.000e-02, eta: 2:33:55, time: 0.659, data_time: 0.007, memory: 5807, loss_rpn_cls: 0.1299, loss_rpn_bbox: 0.0452, s0.loss_cls: 0.3332, s0.acc: 92.9492, s0.loss_bbox: 0.1033, s1.loss_cls: 0.1464, s1.acc: 92.9224, s1.loss_bbox: 0.0915, s2.loss_cls: 0.0608, s2.acc: 94.2481, s2.loss_bbox: 0.0444, loss: 0.9548, grad_norm: 2.8927
2021-09-28 09:21:28,655 - mmdet - INFO - Epoch [1][640/1221]	lr: 2.000e-02, eta: 2:33:50, time: 0.665, data_time: 0.009, memory: 5807, loss_rpn_cls: 0.0930, loss_rpn_bbox: 0.0395, s0.loss_cls: 0.3326, s0.acc: 91.6406, s0.loss_bbox: 0.1341, s1.loss_cls: 0.1599, s1.acc: 91.7432, s1.loss_bbox: 0.1091, s2.loss_cls: 0.0703, s2.acc: 92.6514, s2.loss_bbox: 0.0537, loss: 0.9921, grad_norm: 2.3332
2021-09-28 09:21:34,986 - mmdet - INFO - Epoch [1][650/1221]	lr: 2.000e-02, eta: 2:33:38, time: 0.633, data_time: 0.008, memory: 5807, loss_rpn_cls: 0.1058, loss_rpn_bbox: 0.0528, s0.loss_cls: 0.3786, s0.acc: 90.9375

...

2021-09-28 10:20:57,057 - mmdet - INFO - Epoch [5][1210/1221]	lr: 2.000e-02, eta: 1:33:05, time: 0.660, data_time: 0.008, memory: 5807, loss_rpn_cls: 0.0728, loss_rpn_bbox: 0.0312, s0.loss_cls: 0.3342, s0.acc: 90.0488, s0.loss_bbox: 0.1429, s1.loss_cls: 0.1654, s1.acc: 90.3988, s1.loss_bbox: 0.1265, s2.loss_cls: 0.0797, s2.acc: 90.3739, s2.loss_bbox: 0.0629, loss: 1.0156, grad_norm: 2.9981
2021-09-28 10:21:03,538 - mmdet - INFO - Epoch [5][1220/1221]	lr: 2.000e-02, eta: 1:32:58, time: 0.648, data_time: 0.008, memory: 5807, loss_rpn_cls: 0.0476, loss_rpn_bbox: 0.0354, s0.loss_cls: 0.3002, s0.acc: 91.2109, s0.loss_bbox: 0.1151, s1.loss_cls: 0.1509, s1.acc: 90.7757, s1.loss_bbox: 0.1113, s2.loss_cls: 0.0701, s2.acc: 91.2778, s2.loss_bbox: 0.0602, loss: 0.8907, grad_norm: 2.8060
2021-09-28 10:21:04,268 - mmdet - INFO - Saving checkpoint at 5 epochs
2021-09-28 10:21:15,851 - mmdet - INFO - Epoch [6][10/1221]	lr: 2.000e-02, eta: 1:32:53, time: 0.895, data_time: 0.235, memory: 5807, loss_rpn_

2021-09-28 10:23:21,623 - mmdet - INFO - Epoch [6][200/1221]	lr: 2.000e-02, eta: 1:30:52, time: 0.655, data_time: 0.008, memory: 5807, loss_rpn_cls: 0.0299, loss_rpn_bbox: 0.0182, s0.loss_cls: 0.2123, s0.acc: 93.5059, s0.loss_bbox: 0.0725, s1.loss_cls: 0.1127, s1.acc: 92.7188, s1.loss_bbox: 0.0741, s2.loss_cls: 0.0534, s2.acc: 93.3030, s2.loss_bbox: 0.0458, loss: 0.6188, grad_norm: 2.3663
2021-09-28 10:23:28,346 - mmdet - INFO - Epoch [6][210/1221]	lr: 2.000e-02, eta: 1:30:45, time: 0.672, data_time: 0.008, memory: 5807, loss_rpn_cls: 0.0476, loss_rpn_bbox: 0.0203, s0.loss_cls: 0.2154, s0.acc: 93.6475, s0.loss_bbox: 0.0724, s1.loss_cls: 0.1144, s1.acc: 93.0392, s1.loss_bbox: 0.0717, s2.loss_cls: 0.0576, s2.acc: 92.7822, s2.loss_bbox: 0.0441, loss: 0.6435, grad_norm: 2.5072
2021-09-28 10:23:35,118 - mmdet - INFO - Epoch [6][220/1221]	lr: 2.000e-02, eta: 1:30:39, time: 0.677, data_time: 0.009, memory: 5807, loss_rpn_cls: 0.0481, loss_rpn_bbox: 0.0456, s0.loss_cls: 0.2582, s0.acc: 92.2998

2021-09-28 10:25:39,618 - mmdet - INFO - Epoch [6][410/1221]	lr: 2.000e-02, eta: 1:28:35, time: 0.634, data_time: 0.007, memory: 5807, loss_rpn_cls: 0.0779, loss_rpn_bbox: 0.0263, s0.loss_cls: 0.2478, s0.acc: 92.3633, s0.loss_bbox: 0.0811, s1.loss_cls: 0.1287, s1.acc: 91.6076, s1.loss_bbox: 0.0795, s2.loss_cls: 0.0666, s2.acc: 91.7830, s2.loss_bbox: 0.0491, loss: 0.7569, grad_norm: 3.0704
2021-09-28 10:25:46,117 - mmdet - INFO - Epoch [6][420/1221]	lr: 2.000e-02, eta: 1:28:29, time: 0.650, data_time: 0.008, memory: 5807, loss_rpn_cls: 0.0336, loss_rpn_bbox: 0.0283, s0.loss_cls: 0.2296, s0.acc: 93.3984, s0.loss_bbox: 0.0793, s1.loss_cls: 0.1179, s1.acc: 93.1802, s1.loss_bbox: 0.0820, s2.loss_cls: 0.0558, s2.acc: 93.9609, s2.loss_bbox: 0.0406, loss: 0.6673, grad_norm: 2.9176
2021-09-28 10:25:52,948 - mmdet - INFO - Epoch [6][430/1221]	lr: 2.000e-02, eta: 1:28:23, time: 0.683, data_time: 0.008, memory: 5807, loss_rpn_cls: 0.0551, loss_rpn_bbox: 0.0324, s0.loss_cls: 0.2717, s0.acc: 92.3047

2021-09-28 10:27:57,650 - mmdet - INFO - Epoch [6][620/1221]	lr: 2.000e-02, eta: 1:26:19, time: 0.641, data_time: 0.007, memory: 5807, loss_rpn_cls: 0.0234, loss_rpn_bbox: 0.0203, s0.loss_cls: 0.1963, s0.acc: 94.2383, s0.loss_bbox: 0.0639, s1.loss_cls: 0.1022, s1.acc: 93.6666, s1.loss_bbox: 0.0617, s2.loss_cls: 0.0521, s2.acc: 93.6076, s2.loss_bbox: 0.0381, loss: 0.5580, grad_norm: 2.3243
2021-09-28 10:28:04,134 - mmdet - INFO - Epoch [6][630/1221]	lr: 2.000e-02, eta: 1:26:13, time: 0.648, data_time: 0.008, memory: 5807, loss_rpn_cls: 0.0423, loss_rpn_bbox: 0.0168, s0.loss_cls: 0.2047, s0.acc: 94.1113, s0.loss_bbox: 0.0717, s1.loss_cls: 0.1032, s1.acc: 93.7769, s1.loss_bbox: 0.0635, s2.loss_cls: 0.0513, s2.acc: 93.4131, s2.loss_bbox: 0.0403, loss: 0.5937, grad_norm: 2.7038
2021-09-28 10:28:10,449 - mmdet - INFO - Epoch [6][640/1221]	lr: 2.000e-02, eta: 1:26:06, time: 0.631, data_time: 0.008, memory: 5807, loss_rpn_cls: 0.0421, loss_rpn_bbox: 0.0225, s0.loss_cls: 0.2346, s0.acc: 92.6758

2021-09-28 10:30:15,837 - mmdet - INFO - Epoch [6][830/1221]	lr: 2.000e-02, eta: 1:24:03, time: 0.662, data_time: 0.009, memory: 5807, loss_rpn_cls: 0.0414, loss_rpn_bbox: 0.0272, s0.loss_cls: 0.2548, s0.acc: 92.0996, s0.loss_bbox: 0.0897, s1.loss_cls: 0.1294, s1.acc: 91.8837, s1.loss_bbox: 0.0918, s2.loss_cls: 0.0639, s2.acc: 91.7519, s2.loss_bbox: 0.0577, loss: 0.7560, grad_norm: 3.0837
2021-09-28 10:30:22,424 - mmdet - INFO - Epoch [6][840/1221]	lr: 2.000e-02, eta: 1:23:57, time: 0.659, data_time: 0.008, memory: 5807, loss_rpn_cls: 0.0675, loss_rpn_bbox: 0.0304, s0.loss_cls: 0.2550, s0.acc: 92.6318, s0.loss_bbox: 0.0924, s1.loss_cls: 0.1214, s1.acc: 92.7793, s1.loss_bbox: 0.0947, s2.loss_cls: 0.0555, s2.acc: 93.5592, s2.loss_bbox: 0.0548, loss: 0.7717, grad_norm: 3.0516
2021-09-28 10:30:29,101 - mmdet - INFO - Epoch [6][850/1221]	lr: 2.000e-02, eta: 1:23:50, time: 0.668, data_time: 0.008, memory: 5807, loss_rpn_cls: 0.0461, loss_rpn_bbox: 0.0208, s0.loss_cls: 0.2147, s0.acc: 93.2959

In [ ]:
print('finish')